In [2]:
%reload_ext autoreload
%autoreload 2

# %matplotlib widget
%gui qt

import time
import random
from copy import copy
from tqdm import tqdm
from pathlib import Path
import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.width = 1000

from vrAnalysis import session
from vrAnalysis import functions
from vrAnalysis import analysis 
from vrAnalysis import helpers
from vrAnalysis import fileManagement as fm
from vrAnalysis import database

from vrAnalysis.redgui import redCellGUI as rgui

vrdb = database.vrDatabase()

In [288]:
for fieldName in vrdb.tableData()[0]:
    print(fieldName)

uSessionID
mouseName
sessionDate
sessionID
experimentType
experimentID
variableGain
behavior
imaging
faceCamera
vrEnvironments
headPlateRotation
numPlanes
planeSeparation
pockelsPercentage
objectiveRotation
vrRegistration
suite2p
suite2pQC
redCellQC
sessionQC
scratchJustification
logtime
sessionNotes
suite2pDate
vrRegistrationDate
vrRegistrationError
vrRegistrationException
redCellQCDate


In [6]:
# Immediate next steps for programming work:
# - ROICaT index alignment tools

In [241]:
# Database Requirements: 
# ---------------------
# GUI: db manager
# - show full database -
# - show selected entries (like for example entries with imaging but no suite2p) -
#                --> It would be sweet to make this work where there are some defaults, but then there is also a "filter command" option which you type and goes as the input to the filtering
# - click on entry and do things:
#                --> open file explorer to that session
#                --> do suite2p
#                --> do red cell management
# - update table data? 
# ---------------------
# Operational Commands: 
# - Automatically do suite2p 
# - Check if registration was done before a suite2p update
# - Find way to determine if a suite2p file was updated after the folder was made (indicating that QC was done afterwards) -- so registration should be rerun for imaging data at least
#    -- check if "iscell" is the only thing that was done after registration. In this case, just update the iscell oneData
# ---------------------
# - Add new entry GUI!!!
# ---------------------
# - Check session folders to see if new session is present that isn't registered in the database? 
# ---------------------
# vrExperimentUpdates: 
# -- if edited iscell, will need to re-register the number of ROIs per plane (in "self.values") and reprocess imaging data!
# ---------------------
# vrExperimentRegistration
# -- if error, then note there was an error and append an explanation...!!!
# ---------------------

In [2]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
sessionid = '701'
# vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In [4]:
mouseName = 'ATL012'
dateString = '2023-02-01'
sessionid = '701'

# Load registered vrExperiment
vrexp = session.vrExperiment(mouseName, dateString, sessionid)

In [3]:
mouseName = 'ATL027'
dateString = '2023-08-01'
sessionid = '701'

# Load registered vrExperiment
vrexp = session.vrExperiment(mouseName, dateString, sessionid)

In [ ]:
# # vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.doPreprocessing()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In session: ATL020/2023-05-04/702, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo.trialIdx>0, but this might not be right. There might be smarter ways to determine which trials are 'good' trials...
frame2time had 2 more than suite2p output. This happens sometimes. I don't like it. I think it's because scanimage sends a TTL before starting the frame
Computing red cell features for ATL020/2023-05-04/702... (usually takes 10-20 seconds)
Facecam preprocessing has not been coded yet!
C:\Users\andrew\Documents\localData\ATL020\2023-05-04\702


In [103]:
# Load registered vrExperiment
vrexp = session.vrExperiment(mouseName, dateString, sessionid)

In [18]:
%%time
speedThreshold = 5
omap, smap, lickmap, distvec = functions.getBehaviorMaps(vrexp,speedThreshold=speedThreshold) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = functions.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, speedThreshold=speedThreshold, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = functions.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = functions.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

CPU times: total: 3.77 s
Wall time: 898 ms


In [168]:
# Code block for opening redSelectionGUI for a vrExperiment
vrexp = session.vrExperiment(mouseName, dateString, sessionid)
redCell = session.redCellProcessing(vrexp)
redSelection = rgui.redSelectionGUI(redCell)

Red Cell curation choices are saved for session ATL027/2023-07-19/701
Successfully updated the redCellQC field of the database to True for session ATL027/2023-07-19/701


In [446]:
mouseName = 'ATL023'
dateString = '2023-05-03'
sessionid = '701'
vrexp = session.vrExperiment(mouseName, dateString, sessionid)

In [449]:
data = vrexp.loadone('mpci.roiActivityF')
redidx = vrexp.getRedIdx()
redmanual = vrexp.loadone('mpciROIs.redCellManualAssignments')

In [450]:
data.shape

(9124, 14389)

In [451]:
redidx.shape

(14389,)

In [452]:
redmanual.shape

(2, 14389)

In [ ]:
redmanual.

In [442]:
vrdb.printMiceInSessions(vrdb.iterSessionNeedRedCellQC())

['ATL012']


In [443]:
for ses in vrdb.iterSessionNeedRedCellQC(mouseName='ATL012'):
    print(ses.sessionPrint())

ATL012/2023-01-18/701
ATL012/2023-01-20/701
ATL012/2023-01-20/702
ATL012/2023-01-23/701
ATL012/2023-01-24/703
ATL012/2023-01-25/701
ATL012/2023-01-26/702
ATL012/2023-01-27/701
ATL012/2023-01-30/702
ATL012/2023-02-01/701
ATL012/2023-02-02/701
ATL012/2023-02-06/701
ATL012/2023-02-07/701
ATL012/2023-02-08/702
ATL012/2023-02-09/701
ATL012/2023-02-14/703
ATL012/2023-02-15/701
ATL012/2023-02-16/701
ATL012/2023-02-20/701
ATL012/2023-02-21/702
ATL012/2023-02-22/701
ATL012/2023-02-27/701
ATL012/2023-02-28/701
ATL012/2023-03-01/701
ATL012/2023-03-03/701


In [439]:
mouseName = 'ATL012'
dateString = '2023-02-07'
sessionid = '701'

# Load registered vrExperiment
vrexp = session.vrExperiment(mouseName, dateString, sessionid)
redCell = session.redCellProcessing(vrexp)
redSelection = rgui.redSelectionGUI(redCell, init_yzoom=10)

Red Cell curation choices are saved for session ATL023/2023-05-04/701
Successfully updated the redCellQC field of the database to True for session ATL023/2023-05-04/701


In [444]:
for ses in vrdb.iterSessions(mouseName='ATL012', redCellQC=True):
    print(ses.sessionPrint())

In [441]:
import pandas as pd
pd.options.display.width = 1000
rgui.compareFeatureCutoffs(*vrdb.iterSessions(mouseName='ATL012', redCellQC=True), roundValue=3)

                 session         S2P   DotProduct     Pearson PhaseCorrelation
0  ATL023/2023-04-28/702  [nan, nan]  [0.05, nan]  [nan, nan]     [0.084, nan]
1  ATL023/2023-05-03/701  [nan, nan]  [0.05, nan]  [nan, nan]     [0.084, nan]
2  ATL023/2023-05-04/701  [nan, nan]  [0.05, nan]  [nan, nan]     [0.084, nan]


In [431]:
copyCriterionFrom = session.redCellProcessing('ATL0212','2023-02-07','701', autoload=False)
for ses in vrdb.iterSessionNeedRedCellQC(mouseName='ATL012'):
    redCell = session.redCellProcessing(ses, autoload=False)
    redCell.updateFromSession(copyCriterionFrom)

Red Cell curation choices are saved for session ATL023/2023-04-28/702
Red Cell curation choices are saved for session ATL023/2023-05-04/701


In [ ]:
# next step in analysis: 
# Also - using a cutoff point by the hitch in that distribution might ignore true matches that are soma-dendritic, but maybe I can use ROICaT or npix to avoid those? 

In [ ]:
# okay, looking at these clusters, it seems like there is some serious oversplitting going on in suite2p. That sucks!
# I think I should probably try to rerun some suite2p sessions....
# But for now I'll probably do a "merge" procedure... and I honestly think I'll have to overwrite or add a new "modified" suite2p folder with the stat and ops files